In [1]:
#load the relative function from ocr_embedding
import os
import json
import numpy as np



In [2]:
#change the relative path
print(f"The current working directory is: {os.getcwd()}")

os.chdir("../extension/src")

print(f"The current working directory is: {os.getcwd()}")


The current working directory is: /Users/apple/Documents/GitHub/Rag/RAGApril/notebooks
The current working directory is: /Users/apple/Documents/GitHub/Rag/RAGApril/extension/src


In [3]:
from ocr_embedding import HybridContextualRAG


/Users/apple/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#change the relative path
print(f"The current working directory is: {os.getcwd()}")

os.chdir("../../data")

print(f"The current working directory is: {os.getcwd()}")

The current working directory is: /Users/apple/Documents/GitHub/Rag/RAGApril/extension/src
The current working directory is: /Users/apple/Documents/GitHub/Rag/RAGApril/data


In [5]:
#load the questions, answers, and expected citations,key passages

def load_samples_from_jsonl(jsonl_path, n=None):
    """
    Load all questions, ideal answers, citations, key_passages, and source_files from a JSONL file.
    Args:
        jsonl_path: Path to the JSONL file
        n: Number of samples to load. If None, load all samples.
    Returns:
        A list of dictionaries containing the questions, ideal answers, citations, key_passages, and source_files.
    """
    samples = []
    with open(jsonl_path, 'r') as f:
        for line in f:
            sample = json.loads(line)
            samples.append(sample)
            if n is not None and len(samples) >= n:
                break
    return samples

#load the samples
samples = load_samples_from_jsonl("formatted_summary_questions/summary_questions.jsonl",n=5)



In [6]:
#load the data from txt files for embedding
import glob

txt_files = glob.glob("ocr_output/1807.06209v4.txt")
print(f"Found {len(txt_files)} .txt files in data/ocr_output.")

Found 1 .txt files in data/ocr_output.


In [7]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../extension/src/gemini.json"

In [8]:
hybrid_rag = HybridContextualRAG(
        project_id="camels-453517",
        location="us-central1",
        verbose=True,
        contextual_RAG=True #whether to use contextual RAG or not
    )

/Users/apple/Documents/GitHub/Rag/RAGApril/extension/src/ocr_embedding.py:112: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferWindowMemory(


In [9]:

import os
os.makedirs("index", exist_ok=True)

In [10]:
#build the index
documents = hybrid_rag.load_documents_from_files(txt_files)
chunks = hybrid_rag.split_documents(documents)
hybrid_rag._build_contextual_index(chunks)

  Loaded 1 documents from 1807.06209v4.txt
Total .txt documents loaded: 1
Splitting documents into chunks...
Created 843 chunks from 1 documents
Building contextual retrieval index...
Processing 843 chunks from ocr_output/1807.06209v4.txt
  Contextualizing chunk 1
Original: # Planck 2018 results. VI. Cosmological parameters...
Contextualized: Title of a cosmological parameter analysis paper. # Planck 2018 results. VI. Cosmological parameters...
---
  Contextualizing chunk 2
Original: Planck Collaboration: N. Aghanim ${ }^{34}$, Y. Akrami ${ }^{15,57,59}$, M. Ashdown ${ }^{65,5}$, J....
Contextualized: Authors of the Planck 2018 cosmological parameters results paper. Planck Collaboration: N. Aghanim ${ }^{34}$, Y. Akrami ${ }^{15,57,59}$, M. Ashdown ...
---
  Contextualizing chunk 3
Original: . Challino ${ }^{58,65,11}$, H. C. Chiang ${ }^{25,6}$, J. Chluba ${ }^{64}$, L. P. L. Colombo ${ }^...
Contextualized: Authors of the Planck 2018 cosmological parameters results paper. . Challino 

In [11]:
#load the index if it exists
hybrid_rag.load_index()

Loaded hybrid index with 843 chunks


True

In [12]:
os.makedirs("../extension/results/summary_questions_ocr_embedding", exist_ok=True)

In [13]:
#generate the response from the rag.
for idx, sample in enumerate(samples[0:5], 0):
    question = sample['question']
    ideal_answer = sample['ideal']
    expected_citations = sample['citations']
    key_passage = sample['key_passage']
    source_file = sample['source_file']
    print(f"Generating response for question: {question}")
    response = hybrid_rag.answer_question(question)
    print(f"Response: {response}")
  
    # Store each question, ideal, response, expected citations, key passage, source file in a separate json file
    output_path = f"../extension/results/summary_questions_ocr_embedding/ocr_embedding_response_{idx}.json"
    with open(output_path, "w") as f:
        json.dump({
            "question": question,
            "ideal_answer": ideal_answer,
            "generated_answer": response['answer'],
            "expected_citations": expected_citations,
            "key_passage": key_passage,
        }, f, indent=2)

Generating response for question: How is the standard recombination history tested in the Planck 2018 analysis?
[TokenTracker] Sent 17 tokens (total this minute: 9023/200000)
Response: {'answer': 'With precision data from Planck, the physical assumptions of the recombination process can be tested (Hu et al. 1995; Seljak et al. 2003), studying both standard and non-standard physics (Source: 1807.06209v4.txt). The 2018 Planck temperature and improved polarization data provide a more robust analysis (Source: 1807.06209v4.txt).', 'sources': [{'text': 'Section 7.7: Recombination history analysis and findings. In this section, we update the PCP15 search for deviations from the standard recombination history. In particular, improved polarization data ...', 'source': '1807.06209v4.txt', 'score': np.float64(0.697395364943513)}, {'text': 'Discussing deviations from the standard recombination history and the methodology used to analyze them. We use a semi-blind eigen-analysis (often referred to a